# trade_sign_classification

#### Juan Camilo Henao Londono - 13.02.2019
#### AG Guhr - Universitaet Duisburg-Essen

# Classification of trades signs

It is first classified the sign for each trade by comparing the current and the prior price

$$\epsilon (t;n) = \left\{ \begin{array}{cc} \text{sgn}\left(S\left(t;n\right)-S\left(t;n-1\right)\right), & \text{if }S\left(t;n\right)\ne S\left(t;n-1\right)\\ \epsilon\left(t;n-1\right), & otherwise \end{array}\right.$$

If the current price is higher (lower) than the prior price, the trade sign is defined as +1 (-1). If two consecutive trades having the same trading direction together did not exhaust the available volume at the best quote, the prices of both trades would be the same.
 
During the time interval t, the number of trades is denoted by N(t), and the individual trades carried out are numbered by n = 1, ..., N(t). Therefore, it is defined the trade sign for each time interval of one second by

$$\epsilon (t) = \left\{ \begin{array}{cc} \text{sgn}\left(\sum_{n=1}^{N(t)} \epsilon(t;n) \right), & \text{if }N(t) > 0\\ 0, & \text{if } N(t) = 0 \end{array}\right.$$

Here, if more than one trade occur in the one-second interval t, it is averaged all the trade signs in this interval.

As a result, ε(t) = +1 implies that a majority of trades in the time interval t were triggered by buy market orders, whereas ε(t) = −1 indicates a majority of sell market orders. If ε(t) = 0 trades did not take place in the time interval t or there was a balance of buy and sell market orders in this interval.

In [1]:
# Modules

import numpy as np
import os

import gzip
import pickle

__tau__ = 1000

In [2]:
year = '2008'
month = '01'
day = '07'
ticker = 'AAPL'

# Load data

times_, ids_, types_, prices_ = np.loadtxt(gzip.open('../ITCH_{1}/{1}{2}{3}_{0}.csv.gz'
                                 .format(ticker, year, month, day)), usecols=(0, 2, 3, 5),
                                 skiprows=1, dtype=[('col0', 'int16'), ('col1', 'int16'), ('col2', 'U25'), ('col3', 'f8')],
                                 delimiter=',', unpack=True)

# Lists of times, ids, types, volumes and prices
    # List of all the available information available in the data excluding
    # the last two columns

    # List of order types:
    # "B" = 1 - > Add buy order
    # "S" = 2 - > Add sell order
    # "E" = 3 - > Execute outstanding order in part
    # "C" = 4 - > Cancel outstanding order in part
    # "F" = 5 - > Execute outstanding order in full
    # "D" = 6 - > Delete outstanding order in full
    # "X" = 7 - > Bulk volume for the cross event
    # "T" = 8 - > Execute non-displayed order

types_ = (1 * (types_ == 'B') +
         2 * (types_ == 'S') +
         3 * (types_ == 'E') +
         0 * (types_ == 'C') +
         4 * (types_ == 'F') +
         0 * (types_ == 'D') +
         0 * (types_ == 'X') +
         0 * (types_ == 'T'))

ids = ids_[types_ > 0]
times = times_[types_ > 0]
types = types_[types_ > 0]
prices = prices_[types_ > 0]

In [3]:
# Load data

data = np.genfromtxt(gzip.open('../ITCH_{1}/{1}{2}{3}_{0}.csv.gz'
                     .format(ticker, year, month, day)),
                     dtype='str', skip_header=1, delimiter=',')

# Lists of times, ids, types, volumes and prices
# List of all the available information available in the data excluding
# the last two columns

# List of order types:
# "B" = 1 - > Add buy order
# "S" = 2 - > Add sell order
# "E" = 3 - > Execute outstanding order in part
# "C" = 4 - > Cancel outstanding order in part
# "F" = 5 - > Execute outstanding order in full
# "D" = 6 - > Delete outstanding order in full
# "X" = 7 - > Bulk volume for the cross event
# "T" = 8 - > Execute non-displayed order
times_ = np.array([int(mytime) for mytime in data[:, 0]])
ids_ = np.array([int(myid) for myid in data[:, 2]])
types_ = np.array([1 * (mytype == 'B') +
                   2 * (mytype == 'S') +
                   3 * (mytype == 'E') +
                   4 * (mytype == 'C') +
                   5 * (mytype == 'F') +
                   6 * (mytype == 'D') +
                   7 * (mytype == 'X') +
                   8 * (mytype == 'T') for mytype in data[:, 3]])
prices_ = np.array([int(myprice) for myprice in data[:, 5]])

ids = ids_[types_ < 7]
times = times_[types_ < 7]
types = types_[types_ < 7]
prices = prices_[types_ < 7]

In [4]:
# Reference lists
# Reference lists using the original values or the length of the original
# lists

prices_ref = 1 * prices
types_ref = 0 * types
times_ref = 0 * times
newids = {}
hv = 0

# Help lists with the data of the buy orders and sell orders

hv_prices = prices[types < 3]
hv_types = types[types < 3]
hv_times = times[types < 3]

trade_sign = 0 * types
price_sign = 0 * types

In [5]:
# Fill the reference lists where the values of 'T' are 'E', 'C', 'F', 'D'

# For the data in the length of the ids list (all data)
for iii in range(len(ids)):

    # If the data is a sell or buy order
    if (types[iii] < 3):

        # Insert in the dictionary newids a key with the valor of the id
        # and the value of hv (a counter) that is the index in hv_types
        newids[ids[iii]] = hv

        # Increase the value of hv
        hv += 1

        trade_sign[iii] = 0
        price_sign[iii] = 0

    # If the data is not a sell or buy order
    elif (types[iii] == 3 or
            types[iii] == 5):

        # Fill the values of prices_ref with no prices ('E', 'C', 'F', 'D')
        # with the price of the order
        prices_ref[iii] = hv_prices[newids[ids[iii]]]
        
        # Fill the values of types_ref with no  prices ('E', 'C', 'F', 'D')
        # with the type of the order
        types_ref[iii] = hv_types[newids[ids[iii]]]

        # Fill the values of time_ref with no  prices ('E', 'C', 'F', 'D')
        # with the time of the order
        times_ref[iii] = hv_times[newids[ids[iii]]]

        if (hv_types[newids[ids[iii]]] == 2):

            trade_sign[iii] = 1.
            price_sign[iii] = prices_ref[iii]

        elif (hv_types[newids[ids[iii]]] == 1):

            trade_sign[iii] = - 1.
            price_sign[iii] = prices_ref[iii]

    else:

        # Fill the values of types_ref with no  prices ('E', 'C', 'F', 'D')
        # with the type of the order
        types_ref[iii] = hv_types[newids[ids[iii]]]

        # Fill the values of time_ref with no  prices ('E', 'C', 'F', 'D')
        # with the time of the order
        times_ref[iii] = hv_times[newids[ids[iii]]]

        trade_sign[iii] = 0
        price_sign[iii] = 0

In [6]:
# Ordering the data in the open market time

# This line behaves as an or.the two arrays must achieve a condition, in
# this case, be in the market trade hours
day_times_ind = (1. * times / 3600 / 1000 > 9.5) * \
                (1. * times / 3600 / 1000 < 16) > 0

price_signs = price_sign[day_times_ind]
trade_signs = trade_sign[day_times_ind]
times_signs = times[day_times_ind]

In [7]:
# Theoric

count = 0
idx = 0
trade_signs_teo = np.zeros(len(set(times_signs)))
times_signs_teo = np.zeros(len(set(times_signs)))

while (count < len(price_signs)):
        
    signs = trade_signs[count]

    count += 1

    while (count < len(price_signs) and times_signs[count] == times_signs[count - 1]):

        signs += trade_signs[count]
        count += 1

    trade_signs_teo[idx] = np.sign(signs)
    times_signs_teo[idx] = times_signs[count - 1]
    idx += 1

In [8]:
ttt = np.array(list(sorted(set(times_signs))))
print(ttt == times_signs_teo)
print(len(ttt))
print(len(times_signs_teo))
print(ttt)
print(times_signs_teo)

[ True  True  True ...  True  True  True]
1089316
1089316
[34200002 34200005 34200006 ... 57599987 57599989 57599995]
[34200002. 34200005. 34200006. ... 57599987. 57599989. 57599995.]


In [19]:
# Practical
trade_signs_pra = np.zeros(len(set(times_signs)))

for t_idx, t_val in enumerate(sorted(set(times_signs))):
    
    prices_t_equal = price_signs[times_signs == t_val]
    diff_prices = prices_t_equal[1:] - prices_t_equal[0:-1]
    sign_diff_prices = np.sign(diff_prices)
    signs_interval = np.sum(sign_diff_prices)
    trade_signs_pra[t_idx] = np.sign(signs_interval)   


KeyboardInterrupt: 

In [18]:
print(trade_signs_pra)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
len(set(times_signs))

In [ ]:
len(trade_signs_pra) == len(trade_signs_teo)

# Completing the full time entrances

# 34 200 000 ms = 9h30 - 57 600 000 ms = 16h
full_time = np.array(range(34200000, 57600000))

# As there can be several values for the same millisecond, we use the most
# used trade value of each millisecond in the full time array as it
# behaves quiet similar as the original input

count = 0
trade_signs_complete_teo = 0. * full_time
trade_signs_complete_pra = 0. * full_time

for t_idx, t_val in enumerate(full_time):

    most = 0
    sign = 0

    if (count < len(times_signs) and t_val == times_signs[count]):

        most += trade_signs[count]
        sign += np.sign(price_signs[count] - price_signs[count - 1])
        count += 1

        while (count < len(times_signs) and
                times_signs[count - 1] == times_signs[count]):

            most += trade_signs[count]
            sign += np.sign(price_signs[count] - price_signs[count - 1])
            count += 1

        if (most > 0):

            trade_signs_complete_teo[t_idx] = 1.

        elif (most < 0):

            trade_signs_complete_teo[t_idx] = -1.
            
        if (np.sign(sign) > 0):
            
            trade_signs_complete_pra[t_idx] = 1.
            
        elif (np.sign(sign) < 0):
            
            trade_signs_complete_pra[t_idx] = -1. 

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
figure = plt.figure(figsize=(16, 9))

plt.plot(full_time[5399964:5457598] / 1000 / 3600,
         trade_signs_complete_teo[5399964:5457598], '-g',
         label='Trade signs using the majority')
plt.title('Trade signs {}'.format(ticker), fontsize=40)
plt.xlabel(r'Time $[hour]$', fontsize=25)
plt.ylabel(r'Buy $(+1)$ or Sell $(-1)$', fontsize=25)
plt.legend(loc='best', fontsize=20)

In [ ]:
figure = plt.figure(figsize=(16, 9))

plt.plot(full_time[5399964:5457598] / 1000 / 3600,
         trade_signs_complete_pra[5399964:5457598], '-g',
         label='Trade signs using the majority')
plt.title('Trade signs {}'.format(ticker), fontsize=40)
plt.xlabel(r'Time $[hour]$', fontsize=25)
plt.ylabel(r'Buy $(+1)$ or Sell $(-1)$', fontsize=25)
plt.legend(loc='best', fontsize=20)

In [ ]:
np.sum(trade_signs_complete_teo == trade_signs_complete_pra) / len(trade_signs_complete_teo) * 100